In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp_data_analysis.sql import query_sql
import calitp_data_analysis.magics

from shared_utils import gtfs_utils
# import utils

from siuba import *
import pandas as pd


/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Quick example of finding cancelled service in service alerts

In [2]:
alert_query = '''SELECT * FROM `cal-itp-data-infra.staging.stg_rt__service_alerts`
WHERE date = '2022-05-11' AND calitp_itp_id = 300'''

In [3]:
bbb_alerts = query_sql(alert_query) >> distinct(_.id, _keep_all=True)

In [4]:
bbb_alerts.effect.unique()

array([], dtype=object)

In [5]:
no_service = bbb_alerts >> filter(_.effect == 'NO_SERVICE')

In [6]:
# Deduplicated service alerts describing no service
no_service

,calitp_itp_id,calitp_url_number,original_file_path,date,id,active_periods,informed_entities,cause,effect,url_translations,header_text_translations,description_text_translations,tts_header_text_translations,tts_description_text_translations,severity_level,key


In [8]:
# Example of grabbing route_ids from the informed_entities list column
# Warehouse model will need more complex logic (different entities, AND relationships, etc...)
no_service_rts = [entity[0]['routeId'] for entity in no_service.informed_entities.to_list()]

In [10]:
# These are the routes in GTFS Schedule for May 11 that were cancelled via a service alert
bbb_routes = shared_utils.gtfs_utils.get_route_info('2022-05-11', [300])

AttributeError: module 'shared_utils.gtfs_utils' has no attribute 'get_route_info'

In [57]:
bbb_routes >> filter(_.route_id.isin(no_service_rts))

,feed_key,route_key,date,route_continuous_pickup,agency_email,route_short_name,agency_timezone,agency_fare_url,route_desc,route_url,...,agency_phone,route_continuous_drop_off,agency_id,agency_lang,calitp_extracted_at,calitp_itp_id,route_color,route_type,calitp_url_number,calitp_deleted_at
5,4100255323842349957,7100850145565938766,2022-05-11,None,None,44,America/Los_Angeles,None,None,http://bigbluebus.com/Routes-and-Schedules/Rou...,...,310-451-5444,None,6216179,en,2022-03-08,300,E87D1E,3,0,2022-09-01


In [47]:
bbb_trips = shared_utils.gtfs_utils.get_trips('2022-05-11', [300])

In [58]:
# These are the trips in GTFS Schedule for May 11 that were cancelled via a service alert
# Drop these for correct BBB denominator...
bbb_trips >> filter(_.route_id.isin(no_service_rts))

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,...,trip_short_name,calitp_extracted_at,bikes_allowed,trip_headsign,wheelchair_accessible,block_id,direction_id,calitp_hash,shape_id,calitp_deleted_at
10,4100255323842349957,-3713591605883991482,883709,3501,300,0,77710,2022-05-11,0,2022-03-27,...,None,2022-03-29,0,SMC Bundy Campus,0,None,1,Ui3UjFVCRqBn4zR+wux6Hw==,25956,2022-07-18
66,4100255323842349957,-6520643777545136247,883673,3501,300,0,77710,2022-05-11,0,2022-03-27,...,None,2022-03-29,0,SMC Bundy Campus,0,None,1,qlhx+hLbkigzcnMYk6546A==,25956,2022-07-18
69,4100255323842349957,-7391811447841817502,883658,3501,300,0,77710,2022-05-11,0,2022-03-27,...,None,2022-03-29,0,17th St Station,0,None,0,trwQoCUVhXfM+zBKxdJX8Q==,25955,2022-07-18
93,4100255323842349957,-1694631383424735171,883711,3501,300,0,77710,2022-05-11,0,2022-03-27,...,None,2022-03-29,0,SMC Bundy Campus,0,None,1,3+bLxWwiYHTM02jJekWwhg==,25956,2022-07-18
135,4100255323842349957,-856925526770901337,883707,3501,300,0,77710,2022-05-11,0,2022-03-27,...,None,2022-03-29,0,SMC Bundy Campus,0,None,1,vFlZLSfWSJytbwEROjZ8yw==,25956,2022-07-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,4100255323842349957,-4536867871001574744,883704,3501,300,0,40510,2022-05-11,0,2022-03-27,...,None,2022-03-29,0,SMC Bundy Campus,0,None,1,dgf9aDilu7r9eifGBdWAPg==,25956,2022-07-18
1472,4100255323842349957,-3768729868815070839,883648,3501,300,0,77710,2022-05-11,0,2022-03-27,...,None,2022-03-29,0,17th St Station,0,None,0,UJ5APuy718Xg1d6LO0YreA==,25955,2022-07-18
1477,4100255323842349957,-4248367646246635494,883614,3501,300,0,77710,2022-05-11,0,2022-03-27,...,None,2022-03-29,0,17th St Station,0,None,0,+eBNI6FmTd8PEzL9qsHqwQ==,25955,2022-07-18
1510,4100255323842349957,6501687258087098744,883634,3501,300,0,77710,2022-05-11,0,2022-03-27,...,None,2022-03-29,0,17th St Station,0,None,0,RgS6gxy6q31nIfT/f9itpg==,25955,2022-07-18
